In [1]:
#Using Cryptocompare Free API which has historical data for over 90 exchanges, 1800 coins/tokens - 
#CCCAGG is the weighted average price of all exchanges tracked 
#More information - (https://www.cryptocompare.com/coins/guides/how-does-our-cryptocurrecy-index-work/)
#2000 limit of rows

from_symbol = 'BTC'
to_symbol = 'USD'

exchange = 'CCCAGG'
#Intervals supported minute, hour and day
datetime_interval = 'day'

In [11]:
import pandas as pd
import requests
import pickle
from datetime import datetime

#Use plotly and enable offline mode
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [5]:
#Functions to automate naming and saving of csv files from API

def get_filename(from_symbol, to_symbol, exchange, datetime_interval, download_date):
    return '%s_%s_%s_%s_%s.csv' % (from_symbol, to_symbol, exchange, datetime_interval, download_date)


def download_data(from_symbol, to_symbol, exchange, datetime_interval):
    supported_intervals = {'minute', 'hour', 'day'}
    assert datetime_interval in supported_intervals,\
        'datetime_interval should be one of %s' % supported_intervals

    print('Downloading %s trading data for %s %s from %s' %
          (datetime_interval, from_symbol, to_symbol, exchange))
    base_url = 'https://min-api.cryptocompare.com/data/histo'
    url = '%s%s' % (base_url, datetime_interval)

    #Use aggregate to group different periods of time
    params = {'fsym': from_symbol, 'tsym': to_symbol,
              'limit': 2000, 'aggregate': 1,
              'e': exchange}
    request = requests.get(url, params=params)
    data = request.json()
    return data


def convert_to_dataframe(data):
    df = pd.io.json.json_normalize(data, ['Data'])
    df['datetime'] = pd.to_datetime(df.time, unit='s')
    df = df[['datetime', 'low', 'high', 'open',
             'close', 'volumefrom', 'volumeto']]
    return df


def filter_empty_datapoints(df):
    indices = df[df.sum(axis=1) == 0].index
    print('Filtering %d empty datapoints' % indices.shape[0])
    df = df.drop(indices)
    return df


data = download_data(from_symbol, to_symbol, exchange, datetime_interval)
df = convert_to_dataframe(data)
df = filter_empty_datapoints(df)

current_datetime = datetime.now().date().isoformat()
filename = get_filename(from_symbol, to_symbol, exchange, datetime_interval, current_datetime)
print('Saving data to %s' % filename)
df.to_csv(filename, index=False)


Filtering 0 empty datapoints
Saving data to BTC_USD_CCCAGG_day_2018-03-30.csv


In [6]:
def read_dataset(filename):
    print('Reading data from %s' % filename)
    df = pd.read_csv(filename)
    df.datetime = pd.to_datetime(df.datetime) # change type from object to datetime
    df = df.set_index('datetime') 
    df = df.sort_index() # sort by datetime
    print(df.shape)
    return df

df = read_dataset(filename)


Reading data from BTC_USD_CCCAGG_day_2018-03-30.csv
(2001, 6)


In [7]:
df.head()

,low,high,open,close,volumefrom,volumeto
datetime,,,,,,
2012-10-07,11.70,12.62,12.51,11.80,63512.03,762812.96
2012-10-08,10.62,11.95,11.80,11.78,97938.40,1102329.22
2012-10-09,11.57,12.35,11.78,11.90,46768.80,559694.91
2012-10-10,11.77,12.19,11.90,12.12,27722.07,333351.59
2012-10-11,11.86,12.15,12.12,12.03,19831.64,238483.07


In [8]:
df.tail()

,low,high,open,close,volumefrom,volumeto
datetime,,,,,,
2018-03-26,7869.27,8521.04,8472.56,8152.20,131859.97,1.079039e+09
2018-03-27,7752.30,8226.24,8152.18,7808.48,116523.10,9.307550e+08
2018-03-28,7752.60,8117.79,7808.42,7959.78,82590.62,6.577121e+08
2018-03-29,6927.11,7983.77,7959.78,7106.62,185805.88,1.379180e+09
2018-03-30,6595.26,7280.27,7106.62,6733.14,193121.88,1.333667e+09


In [9]:
df.describe()

,low,high,open,close,volumefrom,volumeto
count,2001.000000,2001.000000,2001.000000,2001.000000,2001.000000,2.001000e+03
mean,1406.423618,1536.790370,1476.870580,1480.231664,61194.388376,1.681677e+08
std,2803.063550,3124.319851,2985.032258,2987.154821,59945.102772,4.801793e+08
min,9.740000,10.650000,10.170000,10.170000,404.380000,1.094799e+05
25%,231.590000,239.610000,235.730000,235.770000,21524.090000,3.879198e+06
50%,430.710000,451.540000,442.110000,442.430000,44632.990000,1.530069e+07
75%,840.270000,913.670000,891.070000,892.910000,82901.880000,4.707444e+07
max,18750.910000,19870.620000,19346.600000,19345.490000,572349.320000,6.245732e+09


In [10]:
# Chart the BTC pricing time series
btc_timeseries = go.Scatter(x=df.index, y=df['close'])
py.iplot([btc_timeseries])



Using iPlot allows easy production of interactive charts using D3.js and are easily to explore further by allow you to select time perios and zooming in.

Observations - we can see a spike up in prices around end up 2013 where we see a quick rise in prices and by february 2014 see a over 80% retracement

Prices again start a very high up swing in 2017 reaching a peak in December 2017. Since then we have seen large drop in prices and recently reached approximately 40% of the highs of December 2018. However we are only at levels last seen in November 2017 which was only 4 months ago.

In [12]:
#Bitcoin is the most well knowed cryptocurrency but I want to also explore relationships to alternative coins which I do using poloniex's api

def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df



In [47]:
altcoins = ['ETH','LTC','XRP','BCH','STR','DASH','SC','XMR','XEM']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df


Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1420088400.0&end=1522435804.62233&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1420088400.0&end=1522435804.62233&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1420088400.0&end=1522435804.62233&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_BCH&start=1420088400.0&end=1522435804.62233&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_STR&start=1420088400.0&end=1522435804.62233&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1420088400.0&end=1522435804.62233&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_SC&start=1420088400.0&end=1522435804.62233&period=86400 fro

In [14]:
altcoin_data

{'BCH':                close      high       low      open    quoteVolume  \
 date                                                                
 2017-08-14  0.072788  0.300000  0.000086  0.000086   44412.996773   
 2017-08-15  0.071687  0.077500  0.065000  0.069113   31956.213983   
 2017-08-16  0.068481  0.074154  0.066707  0.071680   18998.145576   
 2017-08-17  0.105000  0.106000  0.066154  0.068481   95334.083856   
 2017-08-18  0.170270  0.197000  0.102500  0.104900  257120.722730   
 2017-08-19  0.178769  0.243000  0.144100  0.170000  256238.481383   
 2017-08-20  0.177517  0.209687  0.164000  0.178769   85968.982113   
 2017-08-21  0.150500  0.185584  0.140000  0.177517   78145.617580   
 2017-08-22  0.168241  0.195000  0.146352  0.150515   79432.899170   
 2017-08-23  0.163186  0.175348  0.154307  0.168241   31646.436201   
 2017-08-24  0.143200  0.168800  0.140100  0.163186   33462.117507   
 2017-08-25  0.144732  0.154218  0.139700  0.143175   24629.039746   
 2017-08-26  

In [53]:
#filter bitcoin dates to same as poloniex alt coin pull
df_polo_dates = df.loc[df.index >= '2015-1-02'] 
df_polo_dates

,low,high,open,close,volumefrom,volumeto
datetime,,,,,,
2015-01-02,313.08,316.40,314.89,315.21,14728.97,4.673972e+06
2015-01-03,284.89,315.83,315.21,287.13,47366.46,1.420956e+07
2015-01-04,255.87,289.94,287.13,264.72,88769.22,2.425539e+07
2015-01-05,262.89,278.32,264.72,274.84,54993.28,1.499538e+07
2015-01-06,271.52,285.11,274.84,282.27,21939.38,6.097640e+06
2015-01-07,280.74,296.06,282.27,291.34,30666.87,8.827164e+06
2015-01-08,278.99,292.71,291.34,282.69,20182.80,5.796312e+06
2015-01-09,277.28,290.59,282.69,287.97,20924.65,5.963204e+06
2015-01-10,271.98,289.25,287.97,273.35,14724.72,4.147357e+06


In [29]:


for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * df_polo_dates.close


In [30]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

# Merge USD price of each altcoin into single dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')
combined_df

,BCH,DASH,ETH,LTC,SC,STR,XEM,XMR,XRP
date,,,,,,,,,
2015-01-02,NaN,1.970696,NaN,2.680527,NaN,0.005526,NaN,0.455163,0.024467
2015-01-03,NaN,1.731506,NaN,2.275680,NaN,0.004873,NaN,0.451351,0.021233
2015-01-04,NaN,1.631551,NaN,1.989418,NaN,0.004326,NaN,0.406369,0.018176
2015-01-05,NaN,1.673138,NaN,2.105335,NaN,0.004936,NaN,0.419593,0.020324
2015-01-06,NaN,1.716408,NaN,2.152825,NaN,0.005072,NaN,0.453297,0.020944
2015-01-07,NaN,1.729112,NaN,2.166978,NaN,0.005125,NaN,0.459359,0.020799
2015-01-08,NaN,1.684304,NaN,2.044338,NaN,0.005416,NaN,0.435730,0.020320
2015-01-09,NaN,1.724644,NaN,2.024420,NaN,0.005716,NaN,0.423261,0.021074
2015-01-10,NaN,1.655891,NaN,1.743579,NaN,0.005278,NaN,0.390527,0.019443


In [31]:
#Function to to create a single line command to generate a graph from a dataframe

def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)


In [35]:
# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (USD) - Log Scale', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale = 'log')

From first glance it looks like prices of alternative currencies are more correlated with each beginning near the middle of 2017

In [36]:
# Calculate the pearson correlation coefficients for cryptocurrencies in 2016 - using %daily change versus absolute #s 
combined_df_2016 = combined_df[combined_df.index.year == 2016]
combined_df_2016.pct_change().corr(method='pearson')




,BCH,DASH,ETH,LTC,SC,STR,XEM,XMR,XRP
BCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DASH,NaN,1.000000,0.220268,0.192503,0.060392,0.164966,0.092035,0.194502,0.206667
ETH,NaN,0.220268,1.000000,0.109131,0.191617,0.117098,0.099350,0.146966,0.176109
LTC,NaN,0.192503,0.109131,1.000000,0.063649,0.252500,0.239174,0.224606,0.232547
SC,NaN,0.060392,0.191617,0.063649,1.000000,0.167524,0.125267,0.068514,0.054366
STR,NaN,0.164966,0.117098,0.252500,0.167524,1.000000,0.273241,0.090271,0.389852
XEM,NaN,0.092035,0.099350,0.239174,0.125267,0.273241,1.000000,0.058195,0.165165
XMR,NaN,0.194502,0.146966,0.224606,0.068514,0.090271,0.058195,1.000000,0.100183
XRP,NaN,0.206667,0.176109,0.232547,0.054366,0.389852,0.165165,0.100183,1.000000


In [40]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').as_matrix(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = 0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)


In [57]:
#Adding in bitcoin for the correlation matrix

combined_with_btc = pd.concat([combined_df,df_polo_dates.close],axis=1)
combined_with_btc = combined_with_btc.rename(columns={'close':'BTC'})
combined_with_btc

,BCH,DASH,ETH,LTC,SC,STR,XEM,XMR,XRP,BTC
2015-01-02,NaN,1.970696,NaN,2.680527,NaN,0.005526,NaN,0.455163,0.024467,315.21
2015-01-03,NaN,1.731506,NaN,2.275680,NaN,0.004873,NaN,0.451351,0.021233,287.13
2015-01-04,NaN,1.631551,NaN,1.989418,NaN,0.004326,NaN,0.406369,0.018176,264.72
2015-01-05,NaN,1.673138,NaN,2.105335,NaN,0.004936,NaN,0.419593,0.020324,274.84
2015-01-06,NaN,1.716408,NaN,2.152825,NaN,0.005072,NaN,0.453297,0.020944,282.27
2015-01-07,NaN,1.729112,NaN,2.166978,NaN,0.005125,NaN,0.459359,0.020799,291.34
2015-01-08,NaN,1.684304,NaN,2.044338,NaN,0.005416,NaN,0.435730,0.020320,282.69
2015-01-09,NaN,1.724644,NaN,2.024420,NaN,0.005716,NaN,0.423261,0.021074,287.97
2015-01-10,NaN,1.655891,NaN,1.743579,NaN,0.005278,NaN,0.390527,0.019443,273.35
2015-01-11,NaN,1.607694,NaN,1.734916,NaN,0.004972,NaN,0.377003,0.018163,264.77


In [63]:
combined_df_2016_btc = combined_with_btc[combined_with_btc.index.year == 2016]

correlation_heatmap(combined_df_2016_btc.pct_change(), "Cryptocurrency Correlations in 2016")
combined_df_2016_btc.pct_change().corr(method='pearson')

,BCH,DASH,ETH,LTC,SC,STR,XEM,XMR,XRP,BTC
BCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DASH,NaN,1.000000,0.220268,0.192503,0.060392,0.164966,0.092035,0.194502,0.206667,0.216470
ETH,NaN,0.220268,1.000000,0.109131,0.191617,0.117098,0.099350,0.146966,0.176109,0.172972
LTC,NaN,0.192503,0.109131,1.000000,0.063649,0.252500,0.239174,0.224606,0.232547,0.829989
SC,NaN,0.060392,0.191617,0.063649,1.000000,0.167524,0.125267,0.068514,0.054366,0.087852
STR,NaN,0.164966,0.117098,0.252500,0.167524,1.000000,0.273241,0.090271,0.389852,0.241982
XEM,NaN,0.092035,0.099350,0.239174,0.125267,0.273241,1.000000,0.058195,0.165165,0.299337
XMR,NaN,0.194502,0.146966,0.224606,0.068514,0.090271,0.058195,1.000000,0.100183,0.231656
XRP,NaN,0.206667,0.176109,0.232547,0.054366,0.389852,0.165165,0.100183,1.000000,0.243788
BTC,NaN,0.216470,0.172972,0.829989,0.087852,0.241982,0.299337,0.231656,0.243788,1.000000


In [62]:
combined_df_2017_btc = combined_with_btc[combined_with_btc.index.year == 2017]

correlation_heatmap(combined_df_2017_btc.pct_change(), "Cryptocurrency Correlations in 2017")
combined_df_2017_btc.pct_change().corr(method='pearson')

,BCH,DASH,ETH,LTC,SC,STR,XEM,XMR,XRP,BTC
BCH,1.000000,0.480873,0.308905,0.222741,0.217139,0.024506,0.187470,0.254298,0.059802,0.072468
DASH,0.480873,1.000000,0.582625,0.419035,0.347706,0.234419,0.408356,0.581108,0.191553,0.473970
ETH,0.308905,0.582625,1.000000,0.486109,0.408488,0.293128,0.460861,0.608007,0.279418,0.506469
LTC,0.222741,0.419035,0.486109,1.000000,0.372584,0.337519,0.423981,0.495404,0.372111,0.492560
SC,0.217139,0.347706,0.408488,0.372584,1.000000,0.420654,0.372832,0.417161,0.283258,0.375130
STR,0.024506,0.234419,0.293128,0.337519,0.420654,1.000000,0.373894,0.362818,0.529077,0.289252
XEM,0.187470,0.408356,0.460861,0.423981,0.372832,0.373894,1.000000,0.410027,0.324917,0.427295
XMR,0.254298,0.581108,0.608007,0.495404,0.417161,0.362818,0.410027,1.000000,0.300031,0.529819
XRP,0.059802,0.191553,0.279418,0.372111,0.283258,0.529077,0.324917,0.300031,1.000000,0.255953
BTC,0.072468,0.473970,0.506469,0.492560,0.375130,0.289252,0.427295,0.529819,0.255953,1.000000


In [64]:
combined_df_2018_btc = combined_with_btc[combined_with_btc.index.year == 2018]
correlation_heatmap(combined_df_2018_btc.pct_change(), "Cryptocurrency Correlations in 2018")
combined_df_2018_btc.pct_change().corr(method='pearson')

,BCH,DASH,ETH,LTC,SC,STR,XEM,XMR,XRP,BTC
BCH,1.000000,0.877111,0.818380,0.777712,0.594802,0.575424,0.535669,0.820220,0.624868,0.857917
DASH,0.877111,1.000000,0.871950,0.869526,0.734408,0.714572,0.683570,0.893050,0.778367,0.906845
ETH,0.818380,0.871950,1.000000,0.769840,0.650680,0.650134,0.679346,0.830954,0.678463,0.836356
LTC,0.777712,0.869526,0.769840,1.000000,0.692478,0.620606,0.559748,0.781648,0.679025,0.853071
SC,0.594802,0.734408,0.650680,0.692478,1.000000,0.647803,0.664264,0.656861,0.729910,0.732648
STR,0.575424,0.714572,0.650134,0.620606,0.647803,1.000000,0.748364,0.670166,0.854022,0.665449
XEM,0.535669,0.683570,0.679346,0.559748,0.664264,0.748364,1.000000,0.643396,0.781174,0.639133
XMR,0.820220,0.893050,0.830954,0.781648,0.656861,0.670166,0.643396,1.000000,0.693810,0.889087
XRP,0.624868,0.778367,0.678463,0.679025,0.729910,0.854022,0.781174,0.693810,1.000000,0.724680
BTC,0.857917,0.906845,0.836356,0.853071,0.732648,0.665449,0.639133,0.889087,0.724680,1.000000


Questions: How are price movements correlated and were there differences over the years? Which coins are most correlated with one another? Are there opportunities to invest in more diverse coins to reduce risk to your overall portfolio?

Observations: Saw much less correlations in 2016 and 2017 versus 2018. In general BTC, LTC, and ETH saw highest correlations most likely due to these 3 coins having the most fiat pairs which people use initially to transfer to other exchanges to buy other alternative currencies. In 2016 and 2017 we also see similar technology/use case coins being more highly correlated such as Dash and Monero which are both privacy coins. Also we see higher correlation Stellar and Ripple which was created by the same co-founder and is envisioned to have similar use cases.

BTC and LTC saw historically very high correlations perhaps due to LTC being marketed as the "Silver" to bitcoin. 

NEM seems to be least correlated with all other coins, consistently across the three years.